In [1]:
%matplotlib inline

In [2]:

import cv2 as cv

#use thinned graph of corner
#iterate over each pixel
    #check neighboring pixels and count how many are "marked"
        #if num neighbors > 2 or num neighbors < 2
            #pixel = node
        #else
            #pixel = edge
def locate_nodes(path_to_image):
#read 3D rbg image as 2D greyscale nparray
    print("be sure you are pathing to a skeleton for locate_nodes")
    blobs = cv.imread(cv.samples.findFile("../Lake_Murray_Map_Skeletons/SE_corner_thinned.png"),0) 
    # last parameter 0 = greyscale 1 = color -1 = unchanged 

    #convert data from int to np bool 
    blobs = blobs > 127 
    #blobs is now a (540,960) 2 D array

    print("(ydim , xdim) == ",blobs.shape)

    ydim = blobs.shape[0] #num rows 540
    xdim = blobs.shape[1] #num columns 960


    # measure some skeleton metrics down here 
    sum_local_neighbors = 0 #number of adjacent skeleton pixels 
    sum_num_nodes = 0 #total number pixels that are of nodes of the skeleton
    sum_num_edges = 0 #total number pixels that are of edges ots 
    sum_num_endpoints = 0 #total number of pixels that are end points of the skeleton
    sum_num_islands = 0 #total number of pixels that are isolated
    sum_points_length = 0 #total number of skeleton pixels aka length of skeleton

    total_num_edgelines = 0 #how many actual edges are there in the network

    #lets make a bunch of arrays storing : node locations, endpoint locations, edge locations,island locations  
    node_locations = []
    edge_locations = []
    endpoint_locations = []
    island_locations = []

    total_skeleton = []
    #here we are going to find the nodes and edges and endpoints

    for row in range(0,ydim): #iterate through rows 
        for col in range(0,xdim): #iterate through columns
            
            if blobs[row][col] == True: #if pixel is skeleton add 1 to length of skeleton
                sum_points_length +=1
                
                #find the sum of the local neighbors included in the skeleton (including self): 1 = solo ,2= endpoint, 3 = edge , 4+ = node 
                for i in range(-1,2):
                    for j in range(-1,2):
                        try: #this because there may be points on the edge of the picture
                            sum_local_neighbors += int(blobs[row +i][col +j])
                        except:
                            pass
                
                #HEY JUST SO YOU KNOW: here I switch formats from (y,x) to (x,y) just to make things more confusing 
                 
                #figure out num nodes endpoints and edges
                if sum_local_neighbors >= 4: #if node
                    node_locations.append([col,row])
                    sum_num_nodes += 1
                    total_num_edgelines += sum_local_neighbors -1 #count connection points (-1 to remove self counting)
                elif sum_local_neighbors == 3: # if edge
                    edge_locations.append([col,row])
                    sum_num_edges += 1
                elif sum_local_neighbors == 2: # if endpoint
                    endpoint_locations.append([col,row])
                    sum_num_endpoints += 1
                    total_num_edgelines += sum_local_neighbors -1 #count connection points (-1 to remove self counting)
                else: #if island 
                    island_locations.append([col,row])
                    print("isolated point at (x,y)", col, row)
                    sum_num_islands +=1
                total_skeleton.append([col,row])
                #reset counter     
                sum_local_neighbors = 0

    #error check
    if sum_num_nodes + sum_num_edges + sum_num_endpoints + sum_num_islands == sum_points_length & len(total_skeleton) == sum_points_length:
        print("No errors detected")
    else:
        print("f*&!")

    total_num_edgelines = total_num_edgelines / 2 #because each edge has two endpoints 
    print("this skeleton has {0} edge lines, {1} nodes, {2} endpoints, and {3} islands. \nthe skeleton is of length {4} pixels.".format(total_num_edgelines , sum_num_nodes, sum_num_endpoints, sum_num_islands, sum_points_length))

    print("the edgelines are of average length {0}, .".format( sum_num_edges / total_num_edgelines))

    print("this function returns total_skeleton, nodes, edges, endpoints, and island's locations in (x,y) coordinate form")
    return total_skeleton,node_locations,edge_locations,edge_locations,island_locations

In [3]:
path = r'../Lake_Murray_Map_Skeletons/SE_corner_thinned.png' 
locate_nodes(path)










be sure you are pathing to a skeleton for locate_nodes
(ydim , xdim) ==  (403, 341)
No errors detected
this skeleton has 315.0 edge lines, 183 nodes, 60 endpoints, and 0 islands. 
the skeleton is of length 3177 pixels.
the edgelines are of average length 9.314285714285715, .
this function returns total_skeleton, nodes, edges, endpoints, and island's locations in (x,y) coordinate form


98, 62],
  [137, 62],
  [156, 62],
  [214, 62],
  [238, 62],
  [270, 62],
  [51, 63],
  [98, 63],
  [137, 63],
  [154, 63],
  [155, 63],
  [239, 63],
  [270, 63],
  [51, 64],
  [98, 64],
  [137, 64],
  [152, 64],
  [153, 64],
  [240, 64],
  [269, 64],
  [52, 65],
  [98, 65],
  [137, 65],
  [150, 65],
  [151, 65],
  [241, 65],
  [269, 65],
  [52, 66],
  [98, 66],
  [137, 66],
  [149, 66],
  [242, 66],
  [269, 66],
  [53, 67],
  [98, 67],
  [137, 67],
  [148, 67],
  [187, 67],
  [242, 67],
  [269, 67],
  [53, 68],
  [98, 68],
  [137, 68],
  [147, 68],
  [187, 68],
  [243, 68],
  [268, 68],
  [54, 69],
  [98, 69],
  [145, 69],
  [146, 69],
  [187, 69],
  [190, 69],
  [243, 69],
  [267, 69],
  [54, 70],
  [98, 70],
  [139, 70],
  [140, 70],
  [141, 70],
  [142, 70],
  [143, 70],
  [144, 70],
  [189, 70],
  [266, 70],
  [55, 71],
  [98, 71],
  [136, 71],
  [187, 71],
  [239, 71],
  [240, 71],
  [241, 71],
  [242, 71],
  [265, 71],
  [55, 72],
  [98, 72],
  [136, 72],
  [187, 72],
  [245, 72